In [ ]:
# Keras 2.0.6 + TensorFlow 1.2.1 GPU
# train part

import pandas as pd
import numpy as np
import random
import glob
import cv2

from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K

K.set_image_dim_ordering('th')

random.seed(42)
original_size = 500
SIZE = 512
smooth = 1.


def double_conv_layer(x, size, dropout, batch_norm):
    axis=1
    conv = Conv2D(size, (3, 3), padding='same')(x)
    
    if batch_norm == True:
        conv = BatchNormalization(axis=axis)(conv)
        
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    
    if batch_norm == True:
        conv = BatchNormalization(axis=axis)(conv)
        
    conv = Activation('relu')(conv)
    
    if dropout > 0:
        conv = Dropout(dropout)(conv)
        
    return conv

def define_model():
    dropout_val = 0
    batch_norm = True
    
    inputs = Input((3, SIZE, SIZE))
    axis=1
        
    conv1 = double_conv_layer(inputs, 32, dropout_val, batch_norm)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = double_conv_layer(pool1, 64, dropout_val, batch_norm)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = double_conv_layer(pool2, 128, dropout_val, batch_norm)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = double_conv_layer(pool3, 256, dropout_val, batch_norm)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = double_conv_layer(pool4, 512, dropout_val, batch_norm)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = double_conv_layer(pool5, 1024, dropout_val, batch_norm)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv5], axis=axis)
    conv7 = double_conv_layer(up6, 512, dropout_val, batch_norm)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv4], axis=axis)
    conv8 = double_conv_layer(up7, 256, dropout_val, batch_norm)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv3], axis=axis)
    conv9 = double_conv_layer(up8, 128, dropout_val, batch_norm)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv9), conv2], axis=axis)
    conv10 = double_conv_layer(up9, 64, dropout_val, batch_norm)
    
    up10 = concatenate([UpSampling2D(size=(2, 2))(conv10), conv1], axis=axis)
    conv11 = double_conv_layer(up10, 32, 0, batch_norm)

    conv12 = Conv2D(1, (1, 1))(conv11)
    conv12 = BatchNormalization(axis=axis)(conv12)
    conv12 = Activation('sigmoid')(conv12)

    model = Model(inputs=inputs, outputs=conv12)
    return model


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def elastic_transform(img, mask, alpha, sigma, random_state=None):
    from scipy.ndimage.interpolation import map_coordinates
    from scipy.ndimage.filters import gaussian_filter
    
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = (img.shape[0], img.shape[1])
    
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
   
    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), indexing='ij')
    indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1))
    
    res_img = np.zeros_like(img)
    res_mask = np.zeros_like(mask)

    
    for i in range(res_img.shape[2]):
        res_img[:,:,i] = map_coordinates(img[:,:,i], indices, order=1).reshape(shape)
        res_img[:,:,i] = cv2.convertScaleAbs(res_img[:,:,i])  
        
    res_mask = map_coordinates(mask, indices, order=1).reshape(shape)
    
    return res_img, res_mask

def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")

    return cv2.LUT(image, table)

def subimage(image, center, theta):
    from math import cos, sin
    
    width, height = SIZE, SIZE
    
    theta *= 3.14159 / 180

    v_x = (cos(theta), sin(theta))
    v_y = (-sin(theta), cos(theta))
    s_x = center[0] - v_x[0] * (width / 2) - v_y[0] * (height / 2)
    s_y = center[1] - v_x[1] * (width / 2) - v_y[1] * (height / 2)

    mapping = np.array([[v_x[0],v_y[0], s_x],
                        [v_x[1],v_y[1], s_y]])

    return cv2.warpAffine(image,mapping,(width, height),flags=cv2.WARP_INVERSE_MAP,borderMode=cv2.BORDER_CONSTANT)


train_list = glob.glob('/data/PathImageSegmentation-data/training/images/*.tif')
train_list = sorted(train_list, key=lambda k: random.random())
mask_list = [s.replace('images/','truth/').replace('.tif','_mask.png') for s in train_list]

# load images into RAM (elastic_transform is too slow)

X_imgs = []
Y_masks = []
X_eimgs = []
Y_emasks = []
bordersize = 6

for iter_ in range(len(train_list)):

    print (iter_)

    img = cv2.imread(train_list[iter_]).astype(np.float32)
    img=cv2.copyMakeBorder(img, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize,
                           borderType= cv2.BORDER_CONSTANT)

    mask = cv2.imread(mask_list[iter_], 0).astype(np.float32)
    mask /= 255.0

    mask=cv2.copyMakeBorder(mask, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize,
                           borderType= cv2.BORDER_CONSTANT)

    for iter2_ in range(0, 3):

        coef_sigma = random.randint(5, 8) * 0.1
        eimg, emask = elastic_transform(img, 255 * mask, mask.shape[1] * 10,
                                          mask.shape[1] * coef_sigma)

        eimg /= 255.0
        eimg -= 0.5

        emask[emask < 100] = 0.
        emask[emask >= 100] = 1.

        X_eimgs.append(eimg)
        Y_emasks.append(emask)

    img /= 255.0
    img -= 0.5

    X_imgs.append(img)
    Y_masks.append(mask)
        
def generate_data(b_size):
    while True:
    
        imgs = []
        masks = []
        
        for s in range(b_size):
                        
            if np.random.random() < 0.5:
                i = random.randint(0, len(train_list) - 1)

                tmp_img = X_imgs[i]
                tmp_mask = Y_masks[i]
            else:
                i = random.randint(0, len(X_eimgs) - 1)
                
                tmp_img = X_eimgs[i]
                tmp_mask = Y_emasks[i]
                
            if np.random.random() < 0.5:     
                gamma = random.randint(5, 20) * 0.1
                tmp_img = adjust_gamma(((tmp_img+0.5) * 255).astype(np.uint8), gamma).astype(np.float32)
                tmp_img /= 255.0
                tmp_img -= 0.5
            
            if np.random.random() < 0.5:
                
                tmp_img = cv2.flip(tmp_img, 0)
                tmp_mask = cv2.flip(tmp_mask,0)
                
            if np.random.random() < 0.5:
                
                tmp_img = cv2.flip(tmp_img, 1)
                tmp_mask = cv2.flip(tmp_mask,1)
                
            if np.random.random() < 0.5:
                
                tmp_img = tmp_img.swapaxes(0, 1)
                tmp_mask = tmp_mask.swapaxes(0, 1)
            
            tmp_mask = tmp_mask[np.newaxis,:,:]
            imgs.append(tmp_img)
            masks.append(tmp_mask)
        
        imgs = np.asarray(imgs)
        masks = np.asarray(masks)
        imgs = np.rollaxis(imgs, 3, 1)
            
        yield (imgs, masks)
            
model = define_model()
model.compile(optimizer=Adam(lr=0.0001), loss=dice_coef_loss, metrics=[dice_coef])

callbacks = [
        EarlyStopping(monitor='val_loss', patience=20, verbose=0),
        ModelCheckpoint('unet_512_tmp.hdf5', monitor='val_loss', save_best_only=True, verbose=0)]

b_size = 4

history = model.fit_generator(generator=generate_data(b_size), steps_per_epoch=200, epochs=200,
                   max_queue_size=10, verbose=1, callbacks=callbacks, validation_steps=50,
                   validation_data=generate_data(b_size))

model.save_weights('unet_512.hdf5')


pd.DataFrame(history.history).to_csv('unet_512_train.csv', index=False)

In [ ]:
# predict part

model = define_model()
model.load_weights('unet_815LB.hdf5')

from skimage.filters import threshold_yen
from skimage import morphology
from scipy import ndimage

def filter_by_size(mask, threshold_size):
    label_im, nb_labels = ndimage.label(mask)
    sizes = ndimage.sum(mask, label_im, range(nb_labels + 1))
    mask_size = sizes < threshold_size
    remove_pixel = mask_size[label_im]
    mask[remove_pixel] = 0
    
    return mask
         
def get_mask(img, model):
    x = []
    x.append(img)
    x = np.asarray(x)
    x = np.rollaxis(x, 3, 1)
    
    y = model.predict(x)
    
    mask = y[0][0]
    mask = mask[6:506,6:506]
    
    thresh = threshold_yen(mask)
    
    mask[mask > thresh] = 1
    mask[mask <= thresh] = 0
    
    return mask.astype(np.uint8)

def union_mask(m):
    #voting
    res = np.zeros_like(m[0])
    
    for i in range(m[0].shape[0]):
        for j in range(m[1].shape[0]):
            
            count = 0
            for idx in range(len(m)):
                count += m[idx][i][j]
                
            if count > 1:
                res[i][j] = 1.0
                
    return res

def pred_mask(img, model):
    img /= 255.0
    img -= 0.5
    bordersize = 6
    img = cv2.copyMakeBorder(img, top=bordersize, bottom=bordersize, left=bordersize, right=bordersize,
                       borderType= cv2.BORDER_CONSTANT)
    
    mask1 = get_mask(img, model)
    img2 = cv2.flip(img, 1)
    img3 = cv2.flip(img, 0)
    img4 = img.swapaxes(0, 1)
    
    mask2 = get_mask(img2, model)
    mask3 = get_mask(img3, model) 
    mask4 = get_mask(img4, model)
   
    mask2 = cv2.flip(mask2, 1)
    mask3 = cv2.flip(mask3, 0)
    mask4 = mask4.swapaxes(0, 1)
    
    m = []
    m.append(mask1)
    m.append(mask2)
    m.append(mask3)
    m.append(mask4)
    mask_u = union_mask(m)
    
    mask_u = morphology.remove_small_holes(mask_u, 600).astype(np.uint8)
    mask_u = filter_by_size(mask_u, 50).astype(np.uint8)
        
    return mask_u

def write_to_txt(path, mask):
    with open(path, 'w') as out:
        for x in range(500):
            for y in range(500):
                out.write(str(int(mask[y, x])))
            out.write('\n')
    
def write_submission():
    test_list = glob.glob('/data/PathImageSegmentation-data/testing/images/*.tif')
    
    for i in tqdm(range(len(test_list))):
                
        img = cv2.imread(test_list[i]).astype(np.float32)
                      
        mask_u = pred_mask(img.copy(), model)
        
        mask_path = test_list[i].replace('/data/PathImageSegmentation-data/testing/images/','')
        mask_path = mask_path.replace('.tif','_mask.txt')
        
        write_to_txt('1/' + mask_path, mask_u)
        
        
write_submission()